Collecting Report's URL from the Web Page

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# set up the driver
driver = webdriver.Firefox()
url = 'https://www.petronas.com/media/reports'
driver.get(url)

# define a function to scroll to the bottom of the page
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5) # wait for 2 seconds to load more content
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height != last_height:
        scroll_to_bottom() # scroll again if more content is loaded

# scroll to the bottom of the page to load all the reports
scroll_to_bottom()

# extract the URLs of all PDF reports
soup = BeautifulSoup(driver.page_source, 'html.parser')
pdf_urls = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]

# print the PDF URLs
print(pdf_urls)

# close the driver
driver.quit()


['/sites/default/files/about-us/petronas-group-of-companies-2021.pdf', '/sites/default/files/about-us/petronas-group-of-companies-2021.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial_Operational%20Report_Q3%20FY2022.pdf',

Analizing Sentiment

In [3]:
import requests
import io
import PyPDF2
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your own Text Analytics API key and endpoint
key = "0cebfaee5b5a4c3a9c4d504320150da0"
endpoint = "https://cwb-petronas-vivien.cognitiveservices.azure.com/"

# Create a Text Analytics client instance
text_analytics_client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

# Define the list of URLs that you want to extract text from
url_list = ['https://www.petronas.com/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf']

# Loop through each URL and extract the text content using PyPDF2
for url in url_list:
    # Download the PDF from the URL
    response = requests.get(url)
    
    # Extract the text content from the PDF
    pdf_file = PyPDF2.PdfReader(io.BytesIO(response.content))
    text = ""
    for page_num in range(len(pdf_file.pages)):
        page = pdf_file.pages[page_num]
        text += page.extract_text()
    print(text)
        
    # Analyze the extracted text using the Text Analytics service
    result = text_analytics_client.analyze_sentiment([text], language="en")
    result = [result]
    # Print the sentiment score and sentiment label for the text
    print(result)
    print(f"Sentiment score: {result.sentiment_scores}")
    print(f"Sentiment label: {result.sentiment}")


PETRONAS
Activity Outlook
2023-2025
Petroliam Nasional Berhad (PETRONAS) (20076-K)
Tower 1, PETRONAS Twin Towers, Kuala Lumpur City Centre
50088 Kuala Lumpur, Malaysia www.petronas.com
Cover Rationale
The global economy is experiencing a 
broad-based and sharper than expected 
slowdown as well as the lingering eﬀects of the 
COVID-19 pandemic, bringing with it labour 
shortages, record-high commodity prices and 
global supply chain disruptions, which will all 
weigh heavily on the outlook.
These challenges, combined with the complexity 
of the energy transition, have necessitated 
industry players to be agile in responding to the 
changing energy landscape and embrace 
innovation as well as new solutions to face the 
new normal towards a lower-carbon future.
PETRONAS
Activity Outlook
2023-2025
Petroliam Nasional Berhad (PETRONAS) (20076-K)
Tower 1, PETRONAS Twin Towers, Kuala Lumpur City Centre
50088 Kuala Lumpur, Malaysia www.petronas.com
Cautionary Statement
This report was developed

AttributeError: 'list' object has no attribute 'sentiment_scores'

Extracting Key Phrases from The First 5 Page

In [14]:
import requests
import io
import PyPDF2
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your own Text Analytics API key and endpoint
key = "0cebfaee5b5a4c3a9c4d504320150da0"
endpoint = "https://cwb-petronas-vivien.cognitiveservices.azure.com/"

# Create a Text Analytics client instance
text_analytics_client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

# Define the list of URLs that you want to extract text from
url_list = ['https://www.petronas.com/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf']

# Loop through each URL and extract the text content using PyPDF2
for url in url_list:
    # Download the PDF from the URL
    response = requests.get(url)
    
    # Extract the text content from the first five pages of the PDF
    pdf_file = PyPDF2.PdfReader(io.BytesIO(response.content))
    text = ""
    for page_num in range(min(1, 5)):
        page = pdf_file.pages[page_num]
        text += page.extract_text()
        print(text)
        
    # Analyze the extracted text using the Text Analytics service
    # result = text_analytics_client.analyze_key_phrases(documents=[text], language="en")
    
    
    # Print the key phrases for the text
    # print(result[0].key_phrases)


PETRONAS
Activity Outlook
2023-2025
Petroliam Nasional Berhad (PETRONAS) (20076-K)
Tower 1, PETRONAS Twin Towers, Kuala Lumpur City Centre
50088 Kuala Lumpur, Malaysia www.petronas.com

